In [1]:
import itertools
import math
import time
import numpy as np  
import function_wmmse_powercontrol as wf

In [2]:
def create_real(num_H, dis, K,seed):
    np.random.seed(seed)
    CH = np.zeros([num_H, K, K]) + 1j*np.zeros([num_H, K, K])
    pos = np.zeros([num_H,2*K,2])
    distance = np.zeros([num_H,K,K])
    Tx = np.arange(K)
    Rx = np.arange(2*K)[K:]

    for i in range(num_H):
        for j in range(len(Tx)):
            pos[i,Tx[j],:] = np.random.uniform(low=[0.0,0.0], high=[100.0, 100.0], size=(2))
            x = pos[i,Tx[j],0]
            y = pos[i,Tx[j],1]
            dice = np.random.uniform(low= dis[0], high = dis[1])
            rad = np.random.uniform(low=0.0, high= 2* np.pi)
            pos[i,Rx[j],:] = [x+dice*np.cos(rad), y + dice*np.sin(rad)] #assume the number of transmitter and receiver are same

        for j in range(K):
            for k in range(K,2*K):
                    d = np.linalg.norm(pos[i,j,:] - pos[i,k,:])

                    L2 = np.sqrt(1/(1+d**2))
                    distance[i,j,k-K] = d  #record the distance 
                    CH[i,j,k-K] = L2 * 1/np.sqrt(2)*(np.random.randn(1)+1j*np.random.randn(1))

                
    return pos, CH, distance

In [3]:
def np_sum_rate(H,p,alpha,var_noise):

    K = H.shape[0]

    H_2 = np.square(H)
    rx_power = np.multiply(H_2, p)

    mask = np.eye(K)
    valid_rx_power = np.sum(np.multiply(rx_power, mask), axis=1)

    interference = np.sum(np.multiply(rx_power, 1 - mask), axis=0) + var_noise

    rate = np.log(1 + np.divide(valid_rx_power, interference))

    w_rate = np.multiply(alpha,rate)
    sum_rate = np.sum(w_rate, axis=1)

    return sum_rate

In [4]:
def optimize_sum_rate(CH):
    num_channels = 3
    #num_users = len(user_powers)
    max_sum_rate = 0
    optimal_allocation = None

    # Generate all possible channel assignments using itertools.product
    for channel_assignment in itertools.product(range(num_channels), repeat=K):
    
        channel_user = []
        for user_idx, channel_idx in enumerate(channel_assignment):
            channel_user.append(channel_idx)
        #find the user index of each channel
        channel_1_index = [(i) for i in range(K) if channel_user[i] == 0]
        channel_2_index = [(i) for i in range(K) if channel_user[i] == 1]
        channel_3_index = [(i) for i in range(K) if channel_user[i] == 2]

        CH1_K = len(channel_1_index)

        CH2_K = len(channel_2_index)
        CH3_K = len(channel_3_index)
        H_CH1 = np.zeros([CH1_K,CH1_K])
        H_CH2 = np.zeros([CH2_K,CH2_K])
        H_CH3 = np.zeros([CH3_K,CH3_K])
 
        for i in range(CH1_K):
            H_CH1[i,i] = CH[channel_1_index[i],channel_1_index[i]]
            for j in range(i):
                H_CH1[i,j] = CH[channel_1_index[i],channel_1_index[j]]
                H_CH1[j,i] = CH[channel_1_index[j],channel_1_index[i]]
     
        for i in range(CH2_K):
            H_CH2[i,i] = CH[channel_2_index[i],channel_2_index[i]]
            for j in range(i):
                H_CH2[i,j] = CH[channel_2_index[i],channel_2_index[j]]
                H_CH2[j,i] = CH[channel_2_index[j],channel_2_index[i]]   
        for i in range(CH3_K):
            H_CH3[i,i] = CH[channel_3_index[i],channel_3_index[i]]
            for j in range(i):
                H_CH3[i,j] = CH[channel_3_index[i],channel_3_index[j]]
                H_CH3[j,i] = CH[channel_3_index[j],channel_3_index[i]] 
 
        Pmax = 1
        Pini_1 = Pmax*np.ones((1,CH1_K,1))
        Pini_2 = Pmax*np.ones((1,CH2_K,1))
        Pini_3 = Pmax*np.ones((1,CH3_K,1))
        alpha_1 = np.ones((1,CH1_K))
        alpha_2 = np.ones((1,CH2_K))
        alpha_3 = np.ones((1,CH3_K))
        
        # using WMMSE to find the power
        Y_1 = wf.batch_WMMSE2(Pini_1,alpha_1,H_CH1,Pmax,var_noise)
        Y_2 = wf.batch_WMMSE2(Pini_2,alpha_2,H_CH2,Pmax,var_noise)
        Y_3 = wf.batch_WMMSE2(Pini_3,alpha_3,H_CH3,Pmax,var_noise)

        
        sum_rate_1 = np_sum_rate(H_CH1,Y_1,alpha_1,var_noise)
        sum_rate_2 = np_sum_rate(H_CH2,Y_2,alpha_2,var_noise)
        sum_rate_3 = np_sum_rate(H_CH3,Y_3,alpha_3,var_noise)
        #print(sum_rate_1)
        #print(sum_rate_2)
        sum_rate = sum_rate_1+sum_rate_2+sum_rate_3
        # Update maximum sum rate and optimal allocation
        if sum_rate > max_sum_rate:
            max_sum_rate = sum_rate
            optimal_allocation = channel_assignment
    
    return max_sum_rate, optimal_allocation

In [7]:
num_H = 10
K=10
testseed = 7
var_noise = 0.001

In [8]:
pos, CH, distance = create_real(num_H,[2,10],K,testseed)
sum_rate = 0
start = time.time()
for i in range(num_H):

    max_sum_rate, optimal_allocation = optimize_sum_rate(abs(CH[i,:,:]))
    sum_rate += max_sum_rate
    print("Maximum sum rate:", max_sum_rate)
    print("Optimal channel allocation:", optimal_allocation)
sum_rate = sum_rate/num_H
end = time.time()
print('GNN time',end-start)

/tmp/ipykernel_26218/1754817176.py:24: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  CH[i,j,k-K] = L2 * 1/np.sqrt(2)*(np.random.randn(1)+1j*np.random.randn(1))


Maximum sum rate: [27.80918575]
Optimal channel allocation: (0, 1, 1, 1, 0, 1, 2, 2, 2, 0)
Maximum sum rate: [28.19679234]
Optimal channel allocation: (0, 1, 1, 0, 2, 1, 2, 0, 2, 1)
Maximum sum rate: [30.14785036]
Optimal channel allocation: (0, 1, 2, 2, 0, 0, 1, 2, 1, 2)
Maximum sum rate: [22.43553075]
Optimal channel allocation: (0, 1, 0, 0, 2, 1, 2, 2, 1, 0)
Maximum sum rate: [31.291161]
Optimal channel allocation: (0, 1, 2, 1, 0, 2, 2, 2, 0, 1)
Maximum sum rate: [25.56188727]
Optimal channel allocation: (0, 1, 2, 0, 0, 2, 1, 2, 2, 1)
Maximum sum rate: [28.51506322]
Optimal channel allocation: (0, 0, 1, 1, 0, 2, 2, 1, 2, 0)
Maximum sum rate: [27.75524236]
Optimal channel allocation: (0, 1, 0, 0, 2, 2, 2, 1, 0, 1)
Maximum sum rate: [21.53230189]
Optimal channel allocation: (0, 2, 2, 0, 1, 2, 0, 1, 0, 2)
Maximum sum rate: [23.36268543]
Optimal channel allocation: (0, 2, 0, 0, 1, 0, 1, 1, 1, 2)
GNN time 457.76978039741516
